In [3]:
import pyarrow.parquet as pq
import pandas as pd
import os

# ---------------------- 1. 基础配置（只改这里！）----------------------
# 你的Parquet根目录（和结果处理器的STORAGE_ROOT一致）
PARQUET_ROOT = r"D:\workspace\xiaoyao\data\stock_minutely_price"
# 要筛选的条件（按需修改）
TARGET_DATE = "20250102"  # 筛选单个日期（如2025年1月2日）
TARGET_STOCK = "002003.XSHE"  # 筛选单个股票（如深交所002001）


# ---------------------- 2. 核心函数：按条件读取数据 ----------------------
def read_filtered_parquet():
    # 第一步：检查目录是否存在
    if not os.path.exists(PARQUET_ROOT):
        print(f"❌ 目录不存在：{PARQUET_ROOT}")
        return None

    # 第二步：构建“符合条件的分区路径”（直接定位到目标目录，避免全量读取）
    # 路径格式：根目录/date=目标日期/stock_code=目标股票
    target_partition = os.path.join(
        PARQUET_ROOT,
        f"date={TARGET_DATE}",  # 日期分区
        f"stock_code={TARGET_STOCK}"  # 股票代码分区（和你修改的分区名一致）
    )

    # 第三步：检查目标分区是否存在
    if not os.path.exists(target_partition):
        print(f"❌ 目标分区不存在：{target_partition}")
        print(f"请确认日期/股票代码是否正确，或目录是否已生成")
        return None

    # 第四步：读取该分区下的Parquet文件
    print(f"✅ 开始读取：{target_partition}")
    # 读取分区内的所有.parquet文件（这里只有data.parquet）
    df = pq.read_table(target_partition).to_pandas()

    # 第五步：简单数据处理（让结果更易读）
    df["datetime"] = df["date"] + " " + df["time"]  # 合并“日期+时间”
    df = df[["datetime", "stock_code", "open", "close", "high", "low", "volume"]]  # 保留关键列
    df = df.sort_values("datetime")  # 按时间排序

    # 第六步：打印结果（验证是否读取成功）
    print(f"\n✅ 读取完成！数据规模：{len(df)} 行 × {len(df.columns)} 列")
    print(f"\n💡 前5行数据预览：")
    print(df.head())

    return df


# ---------------------- 3. 执行读取 ----------------------
if __name__ == "__main__":
    # 调用函数读取数据
    result_df = read_filtered_parquet()

    # （可选）如果读取成功，保存为Excel方便查看
    if result_df is not None and not result_df.empty:
        save_path = f"筛选结果_{TARGET_DATE}_{TARGET_STOCK}.xlsx"
        result_df.to_excel(save_path, index=False)
        print(f"\n💾 结果已保存为Excel：{os.path.abspath(save_path)}")

✅ 开始读取：D:\workspace\xiaoyao\data\stock_minutely_price\date=20250102\stock_code=002003.XSHE

✅ 读取完成！数据规模：240 行 × 7 列

💡 前5行数据预览：
                         datetime   stock_code    open   close    high  \
0  2025-01-02 2025-01-02 09:31:00  002003.XSHE  365.90  366.93  367.44   
1  2025-01-02 2025-01-02 09:32:00  002003.XSHE  367.19  365.13  367.19   
2  2025-01-02 2025-01-02 09:33:00  002003.XSHE  365.13  364.36  365.64   
3  2025-01-02 2025-01-02 09:34:00  002003.XSHE  364.36  364.87  364.87   
4  2025-01-02 2025-01-02 09:35:00  002003.XSHE  365.13  364.61  365.90   

      low  volume  
0  364.36    9469  
1  364.10    7403  
2  364.36    3746  
3  360.24   10637  
4  364.61    6696  

💾 结果已保存为Excel：d:\workspace\xiaoyao\works\tools\筛选结果_20250102_002003.XSHE.xlsx


In [ ]:
# 在D:\workspace\xiaoyao\data\stock_minutely_price下，筛选出2025年10月09日目录下的子目录数量
import os

# 定义目录路径
base_dir = r"D:\workspace\xiaoyao\data\stock_minutely_price"
target_date = "date=20251010"

# 构建目标目录路径
target_dir = os.path.join(base_dir, target_date)

# 检查目录是否存在
if os.path.exists(target_dir) and os.path.isdir(target_dir):
    # 获取子目录列表
    subdirs = [d for d in os.listdir(target_dir) if os.path.isdir(os.path.join(target_dir, d))]
    
    # 输出子目录数量
    print(f"目录 {target_date} 下有 {len(subdirs)} 个子目录")
else:
    print(f"目录 {target_date} 不存在或不是目录")


# 读取文件：D:\workspace\xiaoyao\data\stock_daily_price.parquet，筛选出2025年10月09日的数据，paused==0的股票代码数量
import pandas as pd

# 定义文件路径
file_path = r"D:\workspace\xiaoyao\data\stock_daily_price.parquet"

# 读取Parquet文件
df = pd.read_parquet(file_path)

# 筛选出2025年10月09日的数据
target_date = "2025-10-10"
df = df[df['date'] == target_date]

# 筛选出paused==0的股票代码数量
valid_stock_count = df[df['paused'] == 0]['stock_code'].nunique()

print(f"在 {target_date} 这一天，paused==0 的股票代码数量为 {valid_stock_count}")


目录 date=20251009 下有 5143 个子目录
在 2025-10-09 这一天，paused==0 的股票代码数量为 5143
